### Load Formatter

In [ ]:
%load_ext nb_black

### Load Modules

In [ ]:
import pymc3 as pm
import numpy as np
from scipy import stats
import pandas as pd
from datetime import datetime
import plotly.express as px

### Generate some fake data

In [ ]:
p_true = 0.75

pdf = pd.DataFrame()
for size in [10 ** i for i in range(1, 7)]:
    x = stats.bernoulli.rvs(p=p_true, size=size, random_state=42)
    pdf = pdf.append(pd.DataFrame({"size": size, "x": x}))

### Model with whole data

In [ ]:
x = pdf.x.values[0:100]

with pm.Model() as model:
    p = pm.Beta("p", alpha=1, beta=1)
    obs = pm.Bernoulli("obs", p=p, observed=x)

    trace = pm.sample(1_000, return_inferencedata=False)

In [ ]:
def pandas_bernoulli_model(pdf: pd.DataFrame) -> pd.Series:

    t0 = datetime.now()

    x = pdf.x.values

    with pm.Model() as model:
        p = pm.Beta("p", alpha=1, beta=1)
        obs = pm.Bernoulli("obs", p=p, observed=x)
        trace = pm.sample(1_000, return_inferencedata=False)

    t1 = datetime.now()

    p_hat = trace["p"].mean()
    hdi = pm.hdi(trace["p"], hdi_prob=0.94)
    sample_time = trace.report.t_sampling
    deltat = t1 - t0

    return pd.Series(
        {
            "p_hat": p_hat,
            "hdi_3%": hdi[0],
            "hdi_97%": hdi[1],
            "runtime": deltat,
            "sample_time": sample_time,
        }
    )

In [ ]:
results = pdf.groupby("size").apply(pandas_bernoulli_model)

In [ ]:
results.drop(columns=["runtime"])

In [ ]:
def weighted_bernoulli_loglike(p, weight, value):
    loglike = weight * pm.Bernoulli.dist(p=p).logp(value)
    return loglike


def pandas_bernoulli_model_agg(pdf: pd.DataFrame) -> pd.Series:

    t0 = datetime.now()

    pdf_agg = pdf.assign(freq=1).groupby(["x"]).agg({"freq": "count"}).reset_index()
    weights = pdf_agg.freq.values
    values = pdf_agg.x.values

    with pm.Model() as model:
        p = pm.Beta("p", alpha=1, beta=1)
        obs = pm.Potential(
            "obs", weighted_bernoulli_loglike(p=p, weight=weights, value=values)
        )
        trace = pm.sample(1_000, return_inferencedata=False)
    t1 = datetime.now()

    p_hat = trace["p"].mean()
    hdi = pm.hdi(trace["p"], hdi_prob=0.94)
    sample_time = trace.report.t_sampling
    deltat = t1 - t0

    return pd.Series(
        {
            "p_hat": p_hat,
            "hdi_3%": hdi[0],
            "hdi_97%": hdi[1],
            "runtime": deltat,
            "sample_time": sample_time,
        }
    )

In [ ]:
results_agg = pdf.groupby("size").apply(pandas_bernoulli_model_agg)

In [ ]:
results_agg.drop(columns=["runtime"])

### Poisson Example

In [ ]:
mu_true = 100
pdf_poisson = pd.DataFrame()

for size in [10 ** j for j in range(1, 7)]:
    x = stats.poisson.rvs(mu_true, size=size, random_state=42)
    pdf_poisson = pdf_poisson.append(pd.DataFrame({"size": size, "x": x}))

In [ ]:
def pandas_poisson_model(pdf: pd.DataFrame):

    t0 = datetime.now()

    x = pdf["x"].values

    with pm.Model() as model:
        mu = pm.Gamma("mu", alpha=1, beta=1)
        obs = pm.Poisson("obs", mu=mu, observed=x)
        trace = pm.sample(1_000, return_inferencedata=False)

    t1 = datetime.now()
    deltat = t1 - t0
    mu_hat = trace["mu"].mean()
    sample_time = trace.report.t_sampling
    hdi = pm.hdi(trace["mu"], hdi_prob=0.94)

    return pd.Series(
        {
            "mu_hat": mu_hat,
            "hdi_3%": hdi[0],
            "hdi_97%": hdi[1],
            "runtime": deltat,
            "sample_time": sample_time,
        }
    )

In [ ]:
poisson_results = pdf_poisson.groupby(["size"]).apply(pandas_poisson_model)

In [ ]:
poisson_results.drop(columns=["runtime"])

In [ ]:
def poisson_likelihood(weight, value, mu):
    return weight * pm.Poisson.dist(mu=mu).logp(value)


def pandas_poisson_model_agg(pdf: pd.DataFrame):

    t0 = datetime.now()

    pdf_agg = pdf.assign(freq=1).groupby(["x"]).agg({"freq": "count"}).reset_index()

    values = pdf_agg["x"].values
    weights = pdf_agg["freq"].values

    with pm.Model() as model:
        mu = pm.Gamma("mu", alpha=1, beta=1)
        obs = pm.Potential(
            "obs", poisson_likelihood(weight=weights, value=values, mu=mu)
        )
        trace = pm.sample(1_000, return_inferencedata=False)

    t1 = datetime.now()
    deltat = t1 - t0
    mu_hat = trace["mu"].mean()
    sample_time = trace.report.t_sampling
    hdi = pm.hdi(trace["mu"], hdi_prob=0.94)

    return pd.Series(
        {
            "mu_hat": mu_hat,
            "hdi_3%": hdi[0],
            "hdi_97%": hdi[1],
            "runtime": deltat,
            "sample_time": sample_time,
        }
    )

In [ ]:
poisson_results_agg = pdf_poisson.groupby(["size"]).apply(pandas_poisson_model_agg)

In [ ]:
poisson_results_agg.drop(columns=["runtime"])

### Normal Distribution

In [ ]:
mu_true = (2.5,)
sigma_true = 5

pdf_normal = pd.DataFrame()

for size in [10 ** i for i in range(1, 7)]:

    x = stats.norm.rvs(loc=mu_true, scale=sigma_true, size=size, random_state=42)

    pdf_normal = pdf_normal.append(pd.DataFrame({"size": size, "x": x}))

In [ ]:
def pandas_normal_model(pdf: pd.DataFrame) -> pd.Series:

    t0 = datetime.now()

    x = pdf.x.values

    with pm.Model() as model:
        mu = pm.Normal("mu", mu=0, sigma=1)
        sigma = pm.HalfCauchy("sigma", beta=5)

        obs = pm.Normal("obs", mu=mu, sigma=sigma, observed=x)

        trace = pm.sample(1_000)

    t1 = datetime.now()
    deltat = t1 - t0

    mu_hat = trace["mu"].mean()
    hdi_mu = pm.hdi(trace["mu"], hdi_prob=0.94)

    sigma_hat = trace["sigma"].mean()
    hdi_sigma = pm.hdi(trace["sigma"], hdi_prob=0.94)

    sample_time = trace.report.t_sampling

    return pd.Series(
        {
            "mu_hat": mu_hat,
            "hdi_mu_3%": hdi_mu[0],
            "hdi_mu_97%": hdi_mu[1],
            "sigma_hat": sigma_hat,
            "hdi_sigma_3%": hdi_sigma[0],
            "hdi_sigma_97%": hdi_sigma[1],
            "runtime": deltat,
            "sample_time": sample_time,
        }
    )

In [ ]:
results_normal = pdf_normal.groupby("size").apply(pandas_normal_model)

In [ ]:
results_normal.drop(columns=["runtime"])

In [ ]:
def normal_likelihood(weight, value, mu, sigma):
    return weight * pm.Normal.dist(mu=mu, sigma=sigma).logp(value)


def pandas_normal_model_agg(pdf: pd.DataFrame, bins: int) -> pd.Series:

    t0 = datetime.now()

    pdf_model = (
        pdf.assign(bucket=lambda x: pd.cut(x["x"], bins=100), freq=1)
        .groupby("bucket")
        .agg({"x": "mean", "freq": "count"})
        .dropna()
        .reset_index()
    )

    values = pdf_model["x"].values
    weights = pdf_model["freq"].values

    with pm.Model() as model:
        mu = pm.Normal("mu", mu=0, sigma=1)
        sigma = pm.HalfCauchy("sigma", beta=5)

        obs = pm.Potential(
            "obs", normal_likelihood(weight=weights, value=values, mu=mu, sigma=sigma)
        )

        trace = pm.sample(1_000)

    t1 = datetime.now()
    deltat = t1 - t0

    mu_hat = trace["mu"].mean()
    hdi_mu = pm.hdi(trace["mu"], hdi_prob=0.94)

    sigma_hat = trace["sigma"].mean()
    hdi_sigma = pm.hdi(trace["sigma"], hdi_prob=0.94)

    sample_time = trace.report.t_sampling

    return pd.Series(
        {
            "mu_hat": mu_hat,
            "hdi_mu_3%": hdi_mu[0],
            "hdi_mu_97%": hdi_mu[1],
            "sigma_hat": sigma_hat,
            "hdi_sigma_3%": hdi_sigma[0],
            "hdi_sigma_97%": hdi_sigma[1],
            "runtime": deltat,
            "sample_time": sample_time,
        }
    )

In [ ]:
results_normal_agg = pdf_normal.groupby("size").apply(
    lambda x: pandas_normal_model_agg(x, bins=100)
)

In [ ]:
results_normal_agg.drop(columns=["runtime"])

### Visualizations

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import chart_studio

username = "your_username"
api_key = "your_api"
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

import chart_studio.plotly as py


In [ ]:
results.drop(columns=["runtime"])

In [ ]:
results_agg.drop(columns=["runtime"])

In [ ]:
poisson_results.drop(columns=["runtime"])

In [ ]:
poisson_results_agg.drop(columns=["runtime"])

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
pdf_aux = results.reset_index()
x = pdf_aux["size"].values
y = pdf_aux["p_hat"].values
ylower = pdf_aux["hdi_3%"].values
yupper = pdf_aux["hdi_97%"].values
y2 = pdf_aux["sample_time"].values

fig = fig.add_traces(
    [
        go.Scatter(
            x=np.append(x, x[::-1]),  # x, then x reversed
            y=np.append(yupper, ylower[::-1]),  # upper, then lower reversed
            fill="toself",
            name="HDI Bands (94%)",
            mode="lines",
        ),
        go.Scatter(x=x, y=y, name="Estimate"),
    ]
).add_trace(go.Scatter(x=x, y=y2, name="Sampling Time"), secondary_y=True)

pdf_aux = results_agg.reset_index()
x = pdf_aux["size"].values
y = pdf_aux["p_hat"].values
ylower = pdf_aux["hdi_3%"].values
yupper = pdf_aux["hdi_97%"].values
y2 = pdf_aux["sample_time"].values

fig = fig.add_traces(
    [
        go.Scatter(
            x=np.append(x, x[::-1]),  # x, then x reversed
            y=np.append(yupper, ylower[::-1]),  # upper, then lower reversed
            fill="toself",
            name="HDI Bands (94%) (Agg.)",
            mode="lines",
        ),
        go.Scatter(x=x, y=y, name="Estimate (Agg.)"),
    ]
).add_trace(go.Scatter(x=x, y=y2, name="Sampling Time (Agg.)"), secondary_y=True)

fig = (
    fig.update_xaxes(type="log", title="Sample Size (log10 scale)")
    .update_yaxes(secondary_y=False, title="Estimate")
    .update_yaxes(title="Fitting Time", secondary_y=True)
    .update_layout(title="Bernoulli Model vs Agg. Bernoulli Model")
)
fig.show()

In [ ]:
py.plot(fig, filename="medium-scaling-byi-bernoulli", auto_open=False)

In [ ]:
pdf[pdf["size"] == 1_000].assign(weight=1).groupby("x").agg(
    {"weight": "count"}
).reset_index().rename(columns={"x": "value"})

In [ ]:
results_normal.drop(columns=["runtime"])

In [ ]:
fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{}, {}], [{"colspan": 2}, None]],
    subplot_titles=(
        "Mean Estimates",
        "Standard Deviation Estimates",
        "Sampling Time",
    ),
)

pdf_aux = results_normal.reset_index()
x = pdf_aux["size"].values
mu = pdf_aux["mu_hat"].values
mu_up = pdf_aux["hdi_mu_97%"].values
mu_lo = pdf_aux["hdi_mu_3%"].values
sigma = pdf_aux["sigma_hat"].values
sigma_up = pdf_aux["hdi_sigma_97%"].values
sigma_lo = pdf_aux["hdi_sigma_3%"].values
st = pdf_aux["sample_time"].values

pdf_aux = results_normal_agg.reset_index()
mu_agg = pdf_aux["mu_hat"].values
mu_agg_up = pdf_aux["hdi_mu_97%"].values
mu_agg_lo = pdf_aux["hdi_mu_3%"].values
sigma_agg = pdf_aux["sigma_hat"].values
sigma_agg_up = pdf_aux["hdi_sigma_97%"].values
sigma_agg_lo = pdf_aux["hdi_sigma_3%"].values
st_agg = pdf_aux["sample_time"].values

fig = fig.add_traces(
    [
        go.Scatter(
            x=np.append(x, x[::-1]),  # x, then x reversed
            y=np.append(mu_up, mu_lo[::-1]),  # upper, then lower reversed
            fill="toself",
            name="Mean HDI Bands (94%)",
            mode="lines",
        ),
        go.Scatter(x=x, y=mu, name="Mean Estimate"),
        go.Scatter(
            x=np.append(x, x[::-1]),  # x, then x reversed
            y=np.append(mu_agg_up, mu_agg_lo[::-1]),  # upper, then lower reversed
            fill="toself",
            name="Mean HDI Bands (94%) (Agg.)",
            mode="lines",
        ),
        go.Scatter(x=x, y=mu_agg, name="Mean Estimate (Agg.)"),
        go.Scatter(
            x=np.append(x, x[::-1]),  # x, then x reversed
            y=np.append(sigma_up, sigma_lo[::-1]),  # upper, then lower reversed
            fill="toself",
            name="Std. HDI Bands (94%)",
            mode="lines",
        ),
        go.Scatter(x=x, y=sigma, name="Std. Estimate"),
        go.Scatter(
            x=np.append(x, x[::-1]),  # x, then x reversed
            y=np.append(sigma_agg_up, sigma_agg_lo[::-1]),  # upper, then lower reversed
            fill="toself",
            name="Std. HDI Bands (94%) (Agg.)",
            mode="lines",
        ),
        go.Scatter(x=x, y=sigma_agg, name="Std. Estimate (Agg.)"),
        go.Scatter(x=x, y=st, name="Train Time"),
        go.Scatter(x=x, y=st_agg, name="Train Time (Agg.)"),
    ],
    rows=[1, 1, 1, 1, 1, 1, 1, 1, 2, 2],
    cols=[1, 1, 1, 1, 2, 2, 2, 2, 1, 1],
)

fig = (
    fig.update_xaxes(type="log", title="Sample Size (log10 scale)")
    .update_yaxes(title="Mean", row=1, col=1)
    .update_yaxes(title="Std.", row=1, col=2)
    .update_yaxes(title="Seconds", row=2, col=1)
    .update_layout(title="Normal Model vs Aggregated Normal Model")
)

fig.show()

In [ ]:
py.plot(fig, filename="medium-scaling-byi-normal")

In [ ]:
pdf_normal[pdf_normal["size"] == 1_000].rename(columns={"x": "value"}).assign(
    bins=lambda x: pd.cut(x["value"], bins=10), freq=1
).groupby("bins").agg({"value": "mean", "freq": "count"})

In [ ]:
import seaborn as sns

In [ ]:
sns.displot(data=pdf_normal[pdf_normal["size"] == 1_000], x="x", )